In [1]:
import numpy as np
from numpy.random import permutation
import keras
from keras.utils import np_utils
from keras.models import Model, Sequential 
from keras.layers import Dense, Dropout, Activation, Flatten, Reshape, Merge
from keras.optimizers import SGD, RMSprop
from keras import backend
from keras.layers import Input, merge
from keras.layers.core import Dense, Lambda, Reshape
from keras.layers.convolutional import Conv1D
from keras.models import Model

Using TensorFlow backend.


In [2]:
batch_size = 32
nb_epoch = 4
J = 4 #nb of negative targets
L = 32 #for last Dense layers

In [3]:
# For training
genes_1 = np.load('f_100kout_gene_input.npy') # 100 000 pairs from the head of "input_for_neuronet
mirs_1 = np.load('f_100kout_mir_input.npy')
# Generation of 4 'negative' genes by simply shuffling the original genes' set
perm = permutation(len(genes_1))
genes_01 = genes_1[perm]
print perm, genes_01
perm = permutation(len(genes_1))
genes_02 = genes_1[perm]
perm = permutation(len(genes_1))
genes_03 = genes_1[perm]
perm = permutation(len(genes_1))
genes_04 = genes_1[perm]

[21751 87656 70770 ...,  2760 36038 19659] [[  6.   7.   6. ...,  11.   4.   7.]
 [ 34.  14.  22. ...,  12.   9.  10.]
 [ 27.  19.  17. ...,  11.   7.   7.]
 ..., 
 [ 80.  36.  68. ...,  10.  18.  17.]
 [ 25.  12.  18. ...,   3.   1.   1.]
 [ 87.  25.  44. ...,   9.  10.   8.]]


In [4]:
# For test
genes_1t = np.load('f_10ktail_gene_input.npy') # 10 000 pairs from the tail
mirs_1t = np.load('f_10ktail_mir_input.npy')
perm = permutation(len(genes_1t))
genes_01t = genes_1[perm]
perm = permutation(len(genes_1t))
genes_02t = genes_1[perm]
perm = permutation(len(genes_1t))
genes_03t = genes_1[perm]
perm = permutation(len(genes_1t))
genes_04t = genes_1[perm]

In [5]:
# Positive targets (targets =1 )
genes_branch_p = Sequential()
genes_branch_p.add(Dense(32, input_dim=genes_1.shape[1]))
genes_branch_p.add(Activation('tanh'))
genes_branch_p.add(Dropout(0.5))
genes_branch_p.add(Dense(L))
genes_branch_p.add(Activation('tanh'))
genes_branch_p.add(Dropout(0.5))
# microRNAs
mirs_branch = Sequential()
mirs_branch.add(Dense(32, input_dim=genes_1.shape[1]))
mirs_branch.add(Activation('tanh'))
mirs_branch.add(Dropout(0.5))
mirs_branch.add(Dense(L))
mirs_branch.add(Activation('tanh'))
mirs_branch.add(Dropout(0.5))

In [6]:
# 4 negative targets
genes_branch_n1 = Sequential()
genes_branch_n1.add(Dense(32, input_dim=genes_1.shape[1]))
genes_branch_n1.add(Activation('tanh'))
genes_branch_n1.add(Dropout(0.5))
genes_branch_n1.add(Dense(L))
genes_branch_n1.add(Activation('tanh'))
genes_branch_n1.add(Dropout(0.5))

genes_branch_n2 = Sequential()
genes_branch_n2.add(Dense(32, input_dim=genes_1.shape[1]))
genes_branch_n2.add(Activation('tanh'))
genes_branch_n2.add(Dropout(0.5))
genes_branch_n2.add(Dense(L))
genes_branch_n2.add(Activation('tanh'))
genes_branch_n2.add(Dropout(0.5))

genes_branch_n3 = Sequential()
genes_branch_n3.add(Dense(32, input_dim=genes_1.shape[1]))
genes_branch_n3.add(Activation('tanh'))
genes_branch_n3.add(Dropout(0.5))
genes_branch_n3.add(Dense(L))
genes_branch_n3.add(Activation('tanh'))
genes_branch_n3.add(Dropout(0.5))

genes_branch_n4 = Sequential()
genes_branch_n4.add(Dense(32, input_dim=genes_1.shape[1]))
genes_branch_n4.add(Activation('tanh'))
genes_branch_n4.add(Dropout(0.5))
genes_branch_n4.add(Dense(L))
genes_branch_n4.add(Activation('tanh'))
genes_branch_n4.add(Dropout(0.5))

In [7]:
# R_Q_D= the cosine similarity between microRNAs and genes
R_Q_D_p = Merge([mirs_branch, genes_branch_p], mode = "cos") # See equation (4).
R_Q_D_n1 = Merge([mirs_branch, genes_branch_n1], mode = "cos")
R_Q_D_n2 = Merge([mirs_branch, genes_branch_n2], mode = "cos")
R_Q_D_n3 = Merge([mirs_branch, genes_branch_n3], mode = "cos")
R_Q_D_n4 = Merge([mirs_branch, genes_branch_n4], mode = "cos")
R_Q_D_n = Merge([R_Q_D_n1, R_Q_D_n2, R_Q_D_n3, R_Q_D_n4], mode = "concat")
concat_Rs = Merge([R_Q_D_p, R_Q_D_n], mode = "concat")

model_Rs = Sequential()
model_Rs.add(concat_Rs)
print model_Rs.output_shape
model_Rs.add(Reshape((J + 1, 1)))
print model_Rs.output_shape
weight = np.array([1]).reshape(1, 1, 1,1)
model_Rs.add(Conv1D(1, 1, border_mode = "same", input_shape = (J + 1, 1), activation = "linear", bias = False,
                       weights = [weight]))
model_Rs.add(Reshape((J + 1, )))
print model_Rs.output_shape

(None, 5)
(None, 5, 1)
(None, 5)


In [8]:
model_Rs.add(Lambda(lambda x: backend.softmax(x), output_shape = (J + 1, )))
model_Rs.compile(optimizer = "adadelta", loss = "categorical_crossentropy", metrics=['accuracy'])

In [9]:
y = np.zeros((len(genes_1),J + 1))
y[:,0] = 1
#ne = np.array([mirs_1, genes_1] +[genes_01]+[genes_02]+[genes_03]+[genes_04])
#print ne.shape
model_Rs.fit([mirs_1, genes_1] +[genes_01]+[genes_02]+[genes_03]+[genes_04], y,
             batch_size=batch_size,
              nb_epoch=nb_epoch,
              validation_split=0.1,
            shuffle = True)

Train on 90000 samples, validate on 10000 samples
Epoch 1/4
90000/90000 [==============================] - 28s - loss: 0.1953 - acc: 0.9819 - val_loss: 7.1173e-05 - val_acc: 1.0000
Epoch 2/4
90000/90000 [==============================] - 9s - loss: 0.0144 - acc: 1.0000 - val_loss: 1.9074e-06 - val_acc: 1.0000
Epoch 3/4
90000/90000 [==============================] - 9s - loss: 0.0039 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 4/4
90000/90000 [==============================] - 9s - loss: 0.0013 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 1.0000


In [10]:
# test
yt = np.zeros((len(genes_1t),J + 1))
yt[:,0] = 1
model_Rs.evaluate([mirs_1t, genes_1t] +[genes_01t]+[genes_02t]+[genes_03t]+[genes_04t], yt, batch_size=batch_size)

 9920/10000 [============================>.] - ETA: 0s

[1.1920928955078125e-07, 1.0]